<a href="https://colab.research.google.com/github/DevSehgal/deep-learning-fastai-pytorch/blob/main/FastAI_exercises_20201225.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Creating this notebook to practice the concepts learned in FastAI course until chapter 4

1.   Follow the entire learning process from scratch on MNIST
2.   Replicate the 'bears' example for another set of images - think about an idea



#### Working on the MNIST end to end classifier first
Steps to follow:

1.   Create datasets for both training and validation
2.   Create a simple linear function
3.   Create a loss function
4.   Create a function to calculate gradients, step
5.   Repeat steps until loss is low enough



To understand how PyTorch calculates gradient - check out this article

https://towardsdatascience.com/pytorch-autograd-understanding-the-heart-of-pytorchs-magic-2686cd94ec95

In [ ]:
# Fastbook module contains relevant libraries
!pip install -Uqq fastbook
import fastbook

# To setup fastbook in Colab
fastbook.setup_book()

     |████████████████████████████████| 727kB 12.3MB/s 
     |████████████████████████████████| 1.1MB 49.4MB/s 
     |████████████████████████████████| 194kB 56.2MB/s 
     |████████████████████████████████| 51kB 8.9MB/s 
     |████████████████████████████████| 61kB 10.7MB/s 
Mounted at /content/gdrive


In [ ]:
from fastai.vision.all import *
from fastbook import *

In [ ]:
matplotlib.rc('image', cmap='Greys')

In [ ]:
#Download data to a folder and return the path
path = untar_data(URLs.MNIST_SAMPLE)

### Checking the contents of the data folder and playing around with the image files

In [ ]:
#Path where the data is
print(path)

#Looking at the data folder
print(path.ls())

#Looking into the folders inside the data folder - there are 2 folders called '3' and '7' in 'train' folder and in 'valid' folder
print((path/'train').ls()), print((path/'valid').ls())

#Display names of 2 files in the folder '3'
print((path/'train/3').ls()[0:2])

#Checking number of files in each folder
print(len((path/'train/3').ls()), len((path/'train/7').ls()))
print(len((path/'valid/3').ls()), len((path/'valid/7').ls()))

/root/.fastai/data/mnist_sample
[Path('/root/.fastai/data/mnist_sample/labels.csv'), Path('/root/.fastai/data/mnist_sample/train'), Path('/root/.fastai/data/mnist_sample/valid')]
[Path('/root/.fastai/data/mnist_sample/train/3'), Path('/root/.fastai/data/mnist_sample/train/7')]
[Path('/root/.fastai/data/mnist_sample/valid/3'), Path('/root/.fastai/data/mnist_sample/valid/7')]
[Path('/root/.fastai/data/mnist_sample/train/3/17330.png'), Path('/root/.fastai/data/mnist_sample/train/3/56234.png')]
6131 6265
1010 1028


In [ ]:
#Creating a list of the image files' path

threes = (path/'train/3').ls().sorted()
sevens = (path/'train/7').ls().sorted()

In [ ]:
type(sevens)

fastcore.foundation.L

In [ ]:
Image.open(threes[5])

In [ ]:
img = Image.open(threes[5])

In [ ]:
img

In [ ]:
# Displaying the image as a tensor
tensor(img)

tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,  75, 105, 245, 253, 253, 255, 129,  37,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,  62, 239, 252, 252, 252, 252, 253, 252, 223, 138,   7,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0, 162, 252, 252, 252, 252, 252, 253, 252, 252, 252, 126,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,  40, 217, 252, 139, 132, 132, 133, 160, 252, 252, 212,  

In [ ]:
# Displaying image as a dataframe with color coded cells
im_t = tensor(img)
df = pd.DataFrame(im_t)
df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,75,105,245,253,253,255,129,37,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,62,239,252,252,252,252,253,252,223,138,7,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,162,252,252,252,252,252,253,252,252,252,126,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,40,217,252,139,132,132,133,160,252,252,212,35,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,38,99,2,0,0,0,7,173,252,252,205,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,48,157,252,222,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,120,252,222,0,0,0,0,0,0,0,0,0


In [ ]:
seven_tensors = [tensor(Image.open(o)) for o in sevens]
three_tensors = [tensor(Image.open(o)) for o in threes]

In [ ]:
stacked_sevens = torch.stack(seven_tensors).float()/255
stacked_threes = torch.stack(three_tensors).float()/255

In [ ]:
valid_3_tens = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'3').ls()])
valid_3_tens = valid_3_tens.float()/255

valid_7_tens = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'7').ls()])
valid_7_tens = valid_7_tens.float()/255

In [ ]:
# Displaying image as a dataframe with color coded cells
im_t1 = stacked_sevens[1]
df = pd.DataFrame(im_t1)
df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003922,0.050980,0.027451,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007843,0.364706,0.956863,0.811765,0.031373,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.615686,0.996078,0.772549,0.000000,0.000000,0.000000,0.000000,0.000000,0.070588,0.513726,0.996078,0.996078,0.972549,0.043137,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035294,0.862745,0.996078,0.996078,0.901961,0.407843,0.000000,0.000000,0.000000,0.254902,0.847059,0.996078,0.996078,0.996078,0.854902,0.031373,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.662745,0.996078,0.996078,0.905882,0.494118,0.156863,0.043137,0.274510,0.705882,0.996078,0.996078,0.996078,0.996078,0.996078,0.352941,0.000000,0.000000,0.000000,0.000000


### Creating datasets that are compatible with PyTorch requirements

In [ ]:
# Taking the stacked tensors from the training set, combining them and changing the shape of the combined tensor to a 2D matrix
train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)

In [ ]:
train_x.shape

torch.Size([12396, 784])

In [ ]:
train_y = tensor([0]*len(stacked_threes) + [1]*len(stacked_sevens)).unsqueeze(1)

In [ ]:
train_y.shape

torch.Size([12396, 1])

In [ ]:
# Training dataset

dset = list(zip(train_x,train_y))

In [ ]:
# dset is a list...
type(dset)

list

In [ ]:
#...of tuples...
type(dset[0])

tuple

In [ ]:
#...and each tuple have a tensor...
type(dset[0][0]), type(dset[0][1])

(torch.Tensor, torch.Tensor)

In [ ]:
# Taking the stacked tensors from the training set, combining them and changing the shape of the combined tensor to a 2D matrix
valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1,28*28)

# Labels
valid_y = tensor([0]*len(valid_3_tens)+[1]*len(valid_7_tens)).unsqueeze(1)

#Validation dataset
valid_dset = list(zip(valid_x, valid_y))

Datasets dset and valid_dset are ready for use

In [ ]:
dl = DataLoader(dset, batch_size=256)

In [ ]:
valid_dl = DataLoader(valid_dset, batch_size=256)

### Creating a simple linear function

In [ ]:
def linear1(xb):
  return xb@weights + bias

### Initializing weights and bias

In [ ]:
# Initializing function
def init_params(size, std=1.0): return (torch.randn(size)*std).requires_grad_()

In [ ]:
# Initializing weights and bias
weights = init_params((28*28,1))
bias = init_params(1)

In [ ]:
type(weights), weights.shape, weights[1]

(torch.Tensor,
 torch.Size([784, 1]),
 tensor([1.4873], grad_fn=<SelectBackward>))

In [ ]:
type(bias), bias.shape, bias

(torch.Tensor, torch.Size([1]), tensor([0.3472], requires_grad=True))

#### Trying out matrix multiplication

In [ ]:
train_x.shape

torch.Size([12396, 784])

In [ ]:
weights.shape

torch.Size([784, 1])

In [ ]:
preds = linear1(train_x)
preds

tensor([[ -6.2330],
        [-10.6388],
        [-20.8865],
        ...,
        [-15.9176],
        [ -1.6866],
        [-11.3568]], grad_fn=<AddBackward0>)

In [ ]:
preds.shape

torch.Size([12396, 1])

In [ ]:
train_x@weights + bias

tensor([[ -6.2330],
        [-10.6388],
        [-20.8865],
        ...,
        [-15.9176],
        [ -1.6866],
        [-11.3568]], grad_fn=<AddBackward0>)

### Loss function

In [ ]:
# Loss function takes pred (output of linear1 for an x for current weights and bias)
# and targets (y label corressponding to that x) to calculate loss

def mnist_loss(predictions, targets):
    predictions = predictions.sigmoid()
    return torch.where(targets==1, 1-predictions, predictions).mean()

### Step

In [ ]:
# Initializing weights and bias
weights = init_params((28*28,1))
bias = init_params(1)

In [ ]:
weights[0:10]

tensor([[ 0.1268],
        [-2.4521],
        [ 0.4160],
        [ 1.9025],
        [-1.3525],
        [-0.1303],
        [ 1.7551],
        [ 0.0675],
        [ 0.7402],
        [ 1.4162]], grad_fn=<SliceBackward>)

In [ ]:
weights.mean()

tensor(-0.0128, grad_fn=<MeanBackward0>)

In [ ]:
bias

tensor([-0.2762], requires_grad=True)

In [ ]:
pred = linear1(train_x)
loss = mnist_loss(pred, train_y)
print(loss)
loss.backward()

weights.data = weights - weights.grad
bias.data = bias - bias.grad

tensor(0.5719, grad_fn=<MeanBackward0>)


In [ ]:
weights[0:10]

tensor([[ 0.1268],
        [-2.4521],
        [ 0.4160],
        [ 1.9025],
        [-1.3525],
        [-0.1303],
        [ 1.7551],
        [ 0.0675],
        [ 0.7402],
        [ 1.4162]], grad_fn=<SliceBackward>)

In [ ]:
weights.mean()

tensor(-0.0108, grad_fn=<MeanBackward0>)

In [ ]:
bias

tensor([-0.2566], requires_grad=True)

In [ ]:
weights.grad.zero_()
bias.grad.zero_();

In [ ]:
((pred.sigmoid()>0.5) == train_y).float().mean()

tensor(0.4357)

### Run training steps multiple times

In [ ]:
type(params)

tuple

In [ ]:
s = 1,2

In [ ]:
type(s)

tuple

In [ ]:
a,b = s

In [ ]:
type(b)

int

In [ ]:
# copying as is for now, will experiment
# Runs the validation through the entire dataset but batch wise. For each epoch

def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds>0.5) == yb
    return correct.float().mean()

def validate_epoch(model):
    accs = [batch_accuracy(model(xb), yb) for xb,yb in valid_dl]
    return round(torch.stack(accs).mean().item(), 4)

In [ ]:
# Initializing weights and bias
weights = init_params((28*28,1))
bias = init_params(1)

lr=1

In [ ]:
#Running the process for 10 epochs
for epoch in range(10):
  #For each epoch the training dataset is divided into a number of batches. In the loop below we iterate through those batches
  batch = 0
  print('Validation: ############### ', validate_epoch(linear1))
  for xb, yb in dl:
    #Predict the model output
    pred = linear1(xb)
    print(pred.shape)

    #Calculate loss (to see how far are we from the model we need)
    loss = mnist_loss(pred, yb)
    print(loss)
    
    #Calculate gradients to check how the weights should be reduced
    loss.backward()
    
    #Adjust the weights according to the gradients and the learning rate
    weights.data = weights - weights.grad*lr
    bias.data = bias - bias.grad*lr
    
    print('Epoch: ', epoch)
    print('Batch: ', batch)
    print('Accuracy (training): ',((pred.sigmoid()>0.5) == yb).float().mean())
    batch +=1

    #Reset the gradients to zero
    weights.grad.zero_()
    bias.grad.zero_()
    
    #Rinse and repeat

In [ ]:
print('Validation: ', validate_epoch(linear1))

Validation:  0.9618


Note: **Important learning** - Without mini-batches it takes 100s of epochs to train the model i.e. adjust the weights and bias such that the model gets accurate enough for classification. Instead of using the full data on each pass, we can use mini batches and move along SGD faster so that we can train the model faster and use less GPU compute.

To do so, we will pass the data using dataloader rather passing the entire matrix train_x directly 

### Validate results

Validation accuracy

In [ ]:
pred2 = linear1(valid_x)

In [ ]:
((pred2.sigmoid()>0.5) == valid_y).float().mean()

tensor(0.9612)

In [ ]:
linear1(valid_x[1]).sigmoid()>0.5, valid_y[1]

(tensor([False]), tensor([0]))

### Instead of the loops above, using built-in functions

In [ ]:
dls = DataLoaders(dl, valid_dl)

In [ ]:
simple_net1 = nn.Linear(28*28,1)

In [ ]:
learn1 = Learner(dls, simple_net1, opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

In [ ]:
learn1.fit(10, 1)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.636833,0.503590,0.495584,00:00
1,0.567886,0.197348,0.832188,00:00
2,0.207458,0.176692,0.840530,00:00
3,0.089717,0.105107,0.912169,00:00
4,0.046457,0.077088,0.933759,00:00
5,0.029657,0.061888,0.947007,00:00
6,0.022821,0.052393,0.956330,00:00
7,0.019826,0.046065,0.963199,00:00
8,0.018337,0.041615,0.966143,00:00
9,0.017461,0.038335,0.967615,00:00


In [ ]:
validate_epoch(simple_net1)

0.9677

### Create another model - a simple neural network with 2 layers and a non-linearity in the middle (ReLU in this case)

In [ ]:
dls = DataLoaders(dl, valid_dl)

In [ ]:
simple_net2 = nn.Sequential(
    nn.Linear(28*28,600),
    nn.ReLU(),
    nn.Linear(600,400),
    nn.ReLU(),
    nn.Linear(400,200),
    nn.ReLU(),
    nn.Linear(200,50),
    nn.ReLU(),
    nn.Linear(50,1)
)

In [ ]:
learn2 = Learner(dls, simple_net2, opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

In [ ]:
learn2.fit(30, 1)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.395734,0.493960,0.504416,00:01
1,0.181726,0.409201,0.588813,00:01
2,0.076559,0.120005,0.883709,00:01
3,0.038211,0.078665,0.924926,00:01
4,0.023437,0.060682,0.940628,00:01
5,0.016779,0.055589,0.945044,00:01
6,0.013760,0.050019,0.951423,00:01
7,0.012376,0.044451,0.956330,00:01
8,0.010549,0.039993,0.961727,00:01
9,0.008707,0.033432,0.966634,00:01


In [ ]:
validate_epoch(simple_net2)

0.9907

### Try SOTA - built-in fastai model

In [ ]:
# I do not understand this yet. This is just to draw a parallel between the models above and a SOTA (supposedly) model

dls = ImageDataLoaders.from_folder(path)
learn = cnn_learner(dls, resnet18, pretrained=False, loss_func=F.cross_entropy, metrics=accuracy)
learn.fit_one_cycle(1, 0.1)

epoch,train_loss,valid_loss,accuracy,time
0,0.059540,0.012494,0.996075,00:18
